<a href="https://colab.research.google.com/github/jkworldchampion/Todays_Menu/blob/main/coding_myself_food101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf  
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import defaultdict
from shutil import copy
from PIL import Image

print(tf.__version__)
print(tf.test.gpu_device_name())  # GPU

2.8.2
/device:GPU:0


## 데이터 불러오기

In [2]:
!wget "http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"

--2022-06-09 11:10:20--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2022-06-09 11:10:21--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  8.10MB/s    in 13m 8s  

2022-06-09 11:23:29 (6.05 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [3]:
!tar xzf food-101.tar.gz  # -v 옵션은 과정이 너무 길어.
print("압축 해제 완료")

압축 해제 완료


In [4]:
# trian과 test로 나눈다.
def prepare_data(filepath, src, dest):
  classes_images = defaultdict(list)  # 값을 지정하지 않으면 해당 값의 key값이 리스트로 디폴트된다.
  with open(filepath, 'r') as txt:  # filepath의 파일을 읽기모드로 열어서 txt라고 지명한다.
      paths = [read.strip() for read in txt.readlines()] # paths는 한 줄씩 읽은 데이터를 앞뒤의 \n을 제거한 깔끔한 리스트
      for p in paths:  # 리스트에서 한줄씩 p에 할당
        food = p.split('/')  # '음식이름/번호'의 데이터를 / 로 나누기
        classes_images[food[0]].append(food[1] + '.jpg')  # classes_images리스트에 음식이름에 번호들을 저장.jpg를 붙여서

  for food in classes_images.keys():  # 음식이름이 하나씩 food에 할당
    if not os.path.exists(os.path.join(dest,food)):  # dest/'food'파일이 존재하지 않는다면
      os.makedirs(os.path.join(dest,food))  # 해당 파일을 만든다.
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [5]:
prepare_data('/content/food-101/meta/train.txt', '/content/food-101/images', 'train')

Copying Done!


In [6]:
prepare_data('/content/food-101/meta/test.txt', '/content/food-101/images', 'test')  # 테스트 라는 폴더 만들고, 데이터 넣기

Copying Done!


In [7]:
!find test/apple_pie -type f | wc -w  # 하나의 음식당 test는 250개

250


In [8]:
!find train/apple_pie -type f | wc -w  # train은 750개 합해서 1000개.

750


In [9]:
# 데이터 사이즈를 보기
test_image_1 = Image.open("/content/train/apple_pie/1005649.jpg")
width, height = test_image_1.size
print(width, height)

test_image_2 = Image.open("/content/train/apple_pie/1014775.jpg")
width, height = test_image_2.size
print(width, height)

test_image_3 = Image.open("/content/train/apple_pie/1088809.jpg")
width, height = test_image_3.size
print(width, height)

test_image_4 = Image.open("/content/train/apple_pie/1072416.jpg")
width, height = test_image_4.size
print(width, height)       # 사이즈가 512*512 안쪽인데, 다 다르네?

308 512
512 512
340 512
512 384


### 사진 데이터의 높이, 너비 최솟값 구하기

In [10]:
# 파일 목록 리스트 화
path = "/content/train/apple_pie"
file_list = os.listdir(path)
file_list.sort()
size_data_width = list()
size_data_height = list()

# 크기 정보 리스트로 만들기
count = 0
co = 0
for i in file_list:
    test_image = Image.open(os.path.join(path, i))
    width, height = test_image.size
    size_data_width.append(width)
    size_data_height.append(height)
    if width == 512:
        count+=1
    if height == 512:
        co+=1

# 정렬해서 최소값 확인
size_data_width.sort()
size_data_height.sort()
print(size_data_width)
print("width가 '512'인 개수: ", count)
print(size_data_height)
print("height가 '512'인 개수: ", co)

[289, 289, 306, 308, 308, 339, 340, 340, 341, 382, 382, 382, 382, 382, 382, 382, 382, 382, 382, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 426, 509, 511, 511, 511, 511, 511, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,

### 학습하기

In [11]:
K.clear_session()
n_classes = 101
img_width, img_height = 232, 289   # 최솟값으로 통일
train_data_dir = 'train'   # train data가 있는 디렉토리 경로
validation_data_dir = 'test'
nb_train_samples = 75750     
nb_validation_samples = 25250 
batch_size = 128  

train_datagen = ImageDataGenerator(   # Data Augmentation
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(   # flow_from_directory는 경로만 설정해 주면 알아서 Numpy array Iterator를 생성해준다.
    train_data_dir,   # train data가 있는 경로
    target_size=(img_height, img_width),  # 사이즈 정보 -> 앞에서 설정한 값
    batch_size=batch_size,  
    class_mode='categorical')  # mode는 categorical, sparse, binary가 있다.

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [14]:
resnet = ResNet50(include_top=False, weights=None, classes=101)

In [15]:
x = resnet.output # ??
x

<KerasTensor: shape=(None, None, None, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [16]:
x = GlobalAveragePooling2D()(x)
x

<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'global_average_pooling2d')>

In [17]:
x = Dense(128,activation='relu')(x)
x

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense')>

In [18]:
x = Dropout(0.2)(x)
x

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dropout')>

In [19]:
predictions = Dense(101, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

In [20]:
model = Model(inputs=resnet.input, outputs=predictions)

In [21]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [22]:
checkpointer = ModelCheckpoint(filepath='best_model_class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_class.log')

In [23]:
history = model.fit_generator(train_generator,      # 한 epoch에 1시간30이 소요됨. 너무 시간이 오래걸림...
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_class.hdf5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30
 487/4734 [==>...........................] - ETA: 1:23:09 - loss: 5.2264 - accuracy: 0.0100

KeyboardInterrupt: ignored